In [7]:
import os
import sys
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
folder = "C:\\Users\\celian\\OneDrive\\WorldBank\\Cambodia"

# Try to get inventories from balance sheet information in the business census

In [24]:
businessCensus = pd.read_csv(os.path.join(folder, "Data", "Structured", "BusinessCensus", "businessCensusFull.csv"))

In [85]:
print('Keep firms for which we have inventory data')
col_invent = ['stock_inputs', 'stock_nonproduced_goods', 'stock_produced_goods', 'stock_in_production']
boolindex = (~businessCensus[col_invent].isnull()).any(axis=1) & (businessCensus[col_invent]>0).any(axis=1)
print(boolindex.sum(), "firms over", businessCensus.shape[0])
data = businessCensus[boolindex].copy()

Keep firms for which we have inventory data
416 firms over 505134


In [96]:
# Approach: inventory duration = all types of inventories / sales
data['inventory'] = data[col_invent].sum(axis=1) / data['sales'] * 52
data.loc[data['inventory']>52, 'inventory'] = 52
data.loc[data['sales']==0, 'inventory'] = None

In [97]:
print("Mean inventory:", data['inventory'].mean())
pd.concat([data.groupby('sector_code')['inventory'].mean(), data.groupby('sector_code')['inventory'].count()], axis=1)

Mean inventory: 17.46621939728639


,inventory,inventory
sector_code,,
5,52.000000,2
6,14.536085,9
7,28.244264,7
8,21.720129,4
9,13.520541,117
10,3.068833,1
11,8.942746,6
13,22.312665,5
14,25.493146,2


In [ ]:
# Other useful cols
cols = [
    'nb_workers', 'activity_code', 'sector_code',
    'stock_inputs', 'stock_nonproduced_goods', 'stock_produced_goods', 'stock_in_production',
    'operating_costs', 'operating_revenues', 'other_revenues', 'operating_expenses',
    "costs_produced_goods_sold", "costs_nonproduced_goods_sold", "costs_services_sold"
]

# Use data from Tanzania

In [8]:
def mapISICRev4toSector(series):
    # get activity division, group, section
    isic_class = pd.read_table(os.path.join(folder, "Data", "Structured", "ActivityCodes", "ISICRev4_link.txt"), dtype=str)
    series = series.astype(str).str.zfill(4)
    activity_group = series.str[:3]
    activity_division = series.str[:2]

    # transform into the sectoral classification used in the model
    ## load file correctly
    corr_ISICRev4_sector = pd.read_csv(os.path.join(folder, "Data", "Structured", "ActivityCodes", "corr_ISICRev4_sector.csv"), dtype=str)
    corr_ISICRev4_sector['sector_id'] = corr_ISICRev4_sector['sector_id'].str.zfill(2)
    corr_ISICRev4_sector['ISIC_Rev4_division'] = corr_ISICRev4_sector['ISIC_Rev4_division'].str.zfill(2)
    corr_ISICRev4_sector['ISIC_Rev4_group'] = corr_ISICRev4_sector['ISIC_Rev4_group'].str.zfill(3)
    ## get mapping for divisions and group
    division_mapping = corr_ISICRev4_sector[~corr_ISICRev4_sector['ISIC_Rev4_division'].isnull()].set_index('ISIC_Rev4_division')['sector_id']
    group_mapping = corr_ISICRev4_sector[corr_ISICRev4_sector['ISIC_Rev4_division'].isnull()].set_index('ISIC_Rev4_group')['sector_id']
    ## do mapping
    sector_code = series.copy()
    sector_code = activity_division.map(division_mapping)
    sector_code[sector_code.isnull()] = activity_group.map(group_mapping)[sector_code.isnull()]
    
    return sector_code

In [9]:
data = pd.read_stata(os.path.join(folder, '..', "Tanzania", "Survey", "Results", 'Firm_TZ_weighted_27February_2019.dta'))

# transform into the sectoral classification used in the model
data['sector_code'] = mapISICRev4toSector(data['icisorg'])
print('check unmapped values:', data['sector_code'].isnull().sum())

check unmapped values: 0


### Load supplier information

In [10]:
# rename columns and treat coded values
data = data.rename(columns={'q4_2_'+str(i):'supplier_'+str(i)+'_sector' for i in range(1,6)})
data = data.rename(columns={'q4_9_'+str(i):'supplier_'+str(i)+'_inventory' for i in range(1,6)})
for i in range(1,6):
    data['supplier_'+str(i)+'_inventory'] = data['supplier_'+str(i)+'_inventory'].replace([8888,9999],np.nan)
data[['supplier_1_sector', "supplier_1_inventory"]].head()

# put all supplier inventory and sector together
stacked_table = pd.concat([data[['sector_code', 'supplier_'+str(i)+'_sector', "supplier_"+str(i)+"_inventory"]].copy().rename(columns={'supplier_'+str(i)+'_sector':'supplier_sector', "supplier_"+str(i)+"_inventory":"supplier_inventory"}) for i in range(1,6)])
stacked_table = stacked_table.dropna()

# transform supplier_sector (name of ISIC Rev4 section) into ISIC section code
isicSectionName_to_Code = pd.read_csv(os.path.join(folder, "Data", "Structured", "ActivityCodes", "ISICRev4_section_forMappingSupplierSurvey.txt"), delimiter="\t", dtype=str)
isicSectionName_to_Code = isicSectionName_to_Code.set_index('Description')['Section']
stacked_table['supplier_section'] = stacked_table['supplier_sector'].map(isicSectionName_to_Code)
print('check unmapped values:', stacked_table['supplier_section'].isnull().sum())

check unmapped values: 0


### Map supplier section code into multiple sector_code

In [11]:
# prepare mapping table from ISIC Rev 4 section code to sector code
isic = pd.read_table(os.path.join(folder, "Data", "Structured", "ActivityCodes", "ISICRev4_link.txt"), dtype=str)
isic = isic[['Section', 'Division', 'Group', "Class"]].drop_duplicates()
isic['sector_code'] = mapISICRev4toSector(isic['Class'])
cols = {'Section':"supplier_section", "sector_code":"supplier_sector_code"}
isic = isic[list(cols.keys())].rename(columns=cols)

# merge
merged_stacked_table = pd.merge(stacked_table, isic, on="supplier_section", how="left")

### Calculate result for combination of (supplier sector code, buyer sector code)

In [22]:
res = merged_stacked_table.groupby(['supplier_sector_code', 'sector_code'])['supplier_inventory'].mean().reset_index()
res.set_index(['sector_code', 'supplier_sector_code']).unstack(level=0).head()

supplier_inventory                                   \
sector_code                          06  07    09      10  11    13   15   
supplier_sector_code                                                       
01                                 16.0 NaN  30.0  23.125 NaN  30.0  3.0   
02                                 16.0 NaN  30.0  23.125 NaN  30.0  3.0   
03                                 16.0 NaN  30.0  23.125 NaN  30.0  3.0   
04                                 16.0 NaN  30.0  23.125 NaN  30.0  3.0   
05                                  NaN NaN   NaN     NaN NaN   NaN  NaN   

                                       ...                                    \
sector_code                 16  18  19 ...   46  47  48  49     50    51  57   
supplier_sector_code                   ...                                     
01                    3.000000 NaN NaN ...  NaN NaN NaN NaN  197.5  30.0 NaN   
02                    3.000000 NaN NaN ...  NaN NaN NaN NaN  197.5  30.0 NaN   
03                    3.000000 NaN NaN ...  NaN NaN NaN NaN  197.5  30.0 NaN   
04                    3.000000 NaN NaN ...  NaN NaN NaN NaN  197.5  30.0 NaN   
05                    8.444444 NaN NaN ...  2.0 NaN NaN NaN    NaN   NaN NaN   

                                   
sector_code           58   59  60  
supplier_sector_code               
01                   NaN  NaN NaN  
02                   NaN  NaN NaN  
03                   NaN  NaN NaN  
04                   NaN  NaN NaN  
05                   NaN  3.0 NaN  

[5 rows x 38 columns]

### Use trigrams

In [29]:
input_folder = os.path.join(folder, 'Data', 'Structured', 'InputOutputTable')
df = pd.read_csv(os.path.join(input_folder, 'sector_trigram.csv'))
dic_new_sector_id = df.set_index('id')['trigram'].to_dict()
dic_new_sector_id['import'] = 'IMP'

res = merged_stacked_table.groupby(['supplier_sector_code', 'sector_code'])['supplier_inventory'].mean().reset_index()
res['supplier_sector_code'] = res['supplier_sector_code'].astype(int).map(dic_new_sector_id)
res['sector_code'] = res['sector_code'].astype(int).map(dic_new_sector_id)

# Add missing values

### Form full matrix

In [59]:
mat = res.set_index(['sector_code', 'supplier_sector_code']).unstack(level=0)
mat.columns = mat.columns.droplevel(0)
print(mat.shape)

(59, 38)


In [60]:
missing_col = list(set(dic_new_sector_id.values()) - set(mat.columns) - set(['IMP']))
missing_row = list(set(dic_new_sector_id.values()) - set(mat.index))
print(missing_col, missing_row)

['RES', 'BET', 'LIV', 'MTR', 'MPH', 'MCO', 'SOC', 'MIN', 'FOR', 'VID', 'WAS', 'ACC', 'MTO', 'FIS', 'CRO', 'INF', 'LIB', 'MBA', 'TEL', 'CST', 'CRE', 'VET'] ['REA', 'IMP']


In [61]:
for row in missing_row:
    mat.loc[row] = None
for col in missing_col:
    mat[col] = None
mat.shape

(61, 60)

### Compute average inventory

In [95]:
av_inventory = mat.mean().mean()
av_inventory_per_input_type = mat.mean(axis=1)
av_inventory_per_buyer_type = mat.mean(axis=0)

In [114]:
# we fill missin values by the average of av_inventory_per_input_type and av_inventory_per_buyer_type
mat_av_inventory_per_input_type = pd.DataFrame(index=mat.columns, columns=av_inventory_per_input_type.index, data=[av_inventory_per_input_type]*len(mat.columns)).transpose()
mat_av_inventory_per_buyer_type = pd.DataFrame(index=av_inventory_per_input_type.index, columns=mat.columns, data=[av_inventory_per_buyer_type]*len(mat.index))

In [133]:
mat_av = (mat_av_inventory_per_input_type + mat_av_inventory_per_buyer_type) / 2
mat_av[mat_av_inventory_per_buyer_type.isnull()] = mat_av_inventory_per_input_type[mat_av_inventory_per_buyer_type.isnull()]
mat_av[mat_av_inventory_per_input_type.isnull()] = mat_av_inventory_per_buyer_type[mat_av_inventory_per_input_type.isnull()]
mat_av[mat_av.isnull()] = av_inventory
mat_av.head()

sector_code,ACF,ADM,AMO,ARC,BUI,CON,EDU,ELE,EMP,FIN,...,MTO,FIS,CRO,INF,LIB,MBA,TEL,CST,CRE,VET
supplier_sector_code,,,,,,,,,,,,,,,,,,,,,
ACC,79.207593,98.577259,81.753968,92.642857,82.142857,71.584585,172.910176,110.542857,113.920635,84.123445,...,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714
ACF,22.106982,41.476647,24.653356,35.542245,25.042245,14.483974,115.809565,53.442245,56.820023,27.022834,...,20.084491,20.084491,20.084491,20.084491,20.084491,20.084491,20.084491,20.084491,20.084491,20.084491
ADM,49.231403,68.601068,51.777778,62.666667,52.166667,41.608395,142.933986,80.566667,83.944444,54.147255,...,74.333333,74.333333,74.333333,74.333333,74.333333,74.333333,74.333333,74.333333,74.333333,74.333333
AMO,50.773070,70.142735,53.319444,64.208333,53.708333,43.150062,144.475653,82.108333,85.486111,55.688922,...,77.416667,77.416667,77.416667,77.416667,77.416667,77.416667,77.416667,77.416667,77.416667,77.416667
ARC,79.207593,98.577259,81.753968,92.642857,82.142857,71.584585,172.910176,110.542857,113.920635,84.123445,...,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714,134.285714


### Fill gaps with average

In [141]:
filled_mat.head()

sector_code,ACF,ADM,AMO,ARC,BUI,CON,EDU,ELE,EMP,FIN,...,MTO,FIS,CRO,INF,LIB,MBA,TEL,CST,CRE,VET
supplier_sector_code,,,,,,,,,,,,,,,,,,,,,
ACC,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACF,7.34375,NaN,NaN,NaN,NaN,NaN,50.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADM,NaN,NaN,NaN,NaN,NaN,NaN,103.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMO,NaN,30.0,NaN,NaN,NaN,NaN,270.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARC,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
filled_mat = mat.copy()
filled_mat = filled_mat.mask(filled_mat.isnull(), mat_av)
filled_mat_week = filled_mat / 7

### Export

In [149]:
invent_table = filled_mat_week.unstack().reset_index()
invent_table = invent_table.rename(columns={"sector_code":"buying_sector", "supplier_sector_code":"input_sector", 0:"inventory_duration_target"})
invent_table.to_csv(os.path.join(folder, 'Data', 'Structured', 'Inventories', "inventory_duration_targets.csv"), index=False)
invent_table.head()

,buying_sector,input_sector,inventory_duration_target
0,ACF,ACC,11.3154
1,ACF,ACF,1.04911
2,ACF,ADM,7.03306
3,ACF,AMO,7.2533
4,ACF,ARC,11.3154
